<a href="https://colab.research.google.com/github/w-dan/MATM/blob/main/MATM-code/data-analysis/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial setup

This section includes installing required libraries and loading environment variables, also making sure GPUs are visible.

### Libraries

In [1]:
# special installation
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet

In [2]:
import importlib.util

def install_if_not_exists(package_name, no_deps=False):
    spec = importlib.util.find_spec(package_name)
    if spec is None:
        print(f"[+] Installing {package_name}...")
        if no_deps is False:
          !pip install {package_name} --quiet
        else:
          !pip install --no-deps {package_name} --quiet
    else:
        print(f"[-] {package_name} is already installed.")

In [3]:
# training
install_if_not_exists("xformers", no_deps=True)
install_if_not_exists("trl", no_deps=True)
install_if_not_exists("peft", no_deps=True)
install_if_not_exists("accelerate", no_deps=True)
install_if_not_exists("bitsandbytes", no_deps=True)

# dataset
install_if_not_exists("pymongo", no_deps=True)

[-] xformers is already installed.
[-] trl is already installed.
[-] peft is already installed.
[-] accelerate is already installed.
[-] bitsandbytes is already installed.
[-] pymongo is already installed.


In [4]:
# core
import os, logging, time, torch
from dotenv import load_dotenv

# dataset
from datasets import Dataset
from pymongo import MongoClient
import pandas as pd

# trl
from trl import SFTTrainer

# models and training
from transformers import TrainingArguments
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### Environment variables and constants

In [5]:
load_dotenv(".env")
CONNECTION_STRING = os.getenv("CONNECTION_STRING")
TOKEN = os.getenv("HUGGINGFACE_TOKEN")
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [6]:
DATABASE_NAME = "APTs"

GPU check (Docker container will sometimes lose GPU visibility)

In [7]:
!nvidia-smi

Thu Jun  6 08:41:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN V                 Off | 00000000:5E:00.0 Off |                  N/A |
| 34%   47C    P8              27W / 250W |      4MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Dataset loading


Initially, we will be building a dataset with raw text and tools. The dataset is stored in a MongoDB database, so we will have to fetch the data:

In [8]:
collection_name = "dataset"

In [9]:
client = MongoClient(CONNECTION_STRING)
db = client[DATABASE_NAME]

collection = db[collection_name]

In [10]:
documents = collection.find()

data = []
for doc in documents:
    corpus = doc.get("corpus", "")
    killchain_phase = doc.get("killchain_phase", "")
    apt_group = doc.get("apt_group", "")
    tactics = doc.get("tactics", [])

    data.append({
        "corpus": corpus,
        "killchain_phase": killchain_phase,
        "apt_group": apt_group,
        "tactics": tactics
    })

df = pd.DataFrame(data)
df.drop(['killchain_phase', 'apt_group'], axis=1, inplace=True)

print(f"[+] Shape: {df.shape}")
df.head()

[+] Shape: (37, 2)


,corpus,tactics
0,decoding network data gh0st rat variant nccgro...,"[Execution, Persistence, Credential Access, Co..."
1,5www .prevailion.com /what-wicked-webs-we-unwe...,"[Credential Access, Defense Evasion, Exploitat..."
2,7www .mandiant.com /resources/blog/lightshift-...,"[Defense Evasion, Privilege Escalation, Execut..."
3,2015/11/24 russian financial cybercrime works ...,"[Credential Access, Exfiltration, Persistence]"
4,9/28/21 4:23 pm foggyw eb argeted nobelium mal...,"[Persistence, Credential Access, Command and C..."


In [12]:
dataset = Dataset.from_pandas(df)
type(dataset)

datasets.arrow_dataset.Dataset

# Preparing model

In [13]:
max_seq_length = 2048 # unsloth supports RoPE Scaling internally
dtype = None # None for auto detection
load_in_4bit = True # 4bit quantization to reduce memory usage. Affects accuracy

# 4bit pre quantized models from unsloth
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA TITAN V. Max memory: 11.774 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Adding LoRA adapters only 1 to 10% of all parameters will need to be updated.

In [14]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # any number > 0 (Suggested 8, 16, 32, 64, 128)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # supports any, but = 0 is optimized
    bias = "none",    # supports any, but = "none" is optimized

    use_gradient_checkpointing = "unsloth", # true or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Preparing data

In [15]:
base_prompt = """Extract tactics from the following text. Tactics represent the "why" of an attack technique or sub-technique. It is a tactical goal or reason for performing an action. Choose one or more from these options
    Reconnaissance, Resource Development, Initial Access, Persistence, Execution, Privilege Escalation, Defense Evasion, Credential Access, Discovery, Lateral Movement, Collection, Command and Control, Exfiltration, Impact
    Extract tactics from the following text.
### Text:
{}

### Response:
{}"""


`EOS_TOKEN` must be added to the tokenized output, otherwise it will return infinite generators.

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [16]:
INSTRUCTION = "Please extract the tactics from the given text."

In [17]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    # instructions = examples["instruction"]
    inputs       = examples["corpus"]
    outputs      = examples["tactics"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = base_prompt.format(input, output) + EOS_TOKEN      # Instruction, Input, Output
        texts.append(text)
    return { "text" : texts, }
pass

In [18]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/37 [00:00<?, ? examples/s]

In [19]:
dataset

Dataset({
    features: ['corpus', 'tactics', 'text'],
    num_rows: 37
})

Define training parameters

In [20]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Map (num_proc=2):   0%|          | 0/37 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 37 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.444200
2,3.517300
3,3.245000
4,3.562900
5,3.393200
6,3.308000
7,3.033000
8,3.477300
9,3.319200
10,2.969200


In [22]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

NameError: name 'start_gpu_memory' is not defined

In [23]:
report = "sofacy also known apt28 fancy bear tsar team highly active proliﬁc apt high volume 0day deployment innovative broad malware set sofacy one top groups monitor report protect 2017 diﬀerent regard private reports subscription customers receive steady stream yara ioc reports sofacy reported apt year high level cyber-espionage activity goes back years 2011-2012 group used relatively tiny implant known sofacy sourface ﬁrst stage malware time similarities old miniduke implants made us believe two groups connected although looks split ways certain point original miniduke group switching cosmicduke implant 2014. division malware consistent deﬁnitive point 2013 sofacy group expanded arsenal added backdoors tools including coreshell splm aka xagent aka chopstick jhuhugit built code carberp sources azzy aka advstoreshell netui eviltoss spans across 4-5 generations others seen quite versions implants relatively widespread point still 2015 noticed another wave attacks took advantage new release azzy implant largely undetected antivirus products new wave attacks included new generation usb stealers deployed sofacy initial versions dating february 2015. appeared geared exclusively towards high proﬁle targets sofacy reported presence dnc network alongside apt29 brought possibly highest level public attention group activities 2016 especially data compromise leaked weaponized later 2016 focus turned towards olympics world anti-doping agency wada court arbitration sports cas individuals servers organizations phished compromised similar vein past cyberberkut activity attackers hid behind anonymous activist groups like anonpoland data victimized organizations similarly leaked weaponized write-up survey notables past year 2017 sofacy activity including targeting technology notes infrastructure one research group 100 global visibility collected data isactivity great february 20 2018 2:00 mpresented accordingly external apt28 reports 2017 darkhotel- style activity europe dealer choice spearphishing interest sit 2017 sofacy activity starts heavy focus nato ukrainian partners coinciding lighter interest central asian targets ﬁnishing second half year heavy focus central asian targets shift east dealer choice beginning 2017 began slow cleanup following dealer choice campaign technical characteristics documented colleagues palo alto several stages end 2016. group spearphished targets several waves flash exploits leading carberp based jhuhugit downloaders stages malware seems many folks log pull emails jan 2017 retrieve dealer choice spearphish throughout waves observed targets provided connection even tangential ukraine nato military diplomatic interests multiple cases sofacy spoofs identity target emails spearphish targets interest often military military- technology manufacturing related dealerschoice spearphish nato related global reach coincided focus nato ukraine overstated ksn data showed spearphishing targets geolocated across globe 2017. az fr de iq kg ch ua us vn dealerschoice emails like one able recover third party sources provided additional targeting insight conﬁrmed targeting within ksn data tr pl ba az kr lv ge lv au se 0day deployment sofacy kicked oﬀ year deploying two 0day spearphish document microsoft oﬃce encapsulated postscript type confusion exploit abusing cve-2017-0262 escalation privilege use-after-free exploit abusing cve-2017-0263 group attempted deploy spearphish attachment push small 30kb backdoor known gamefish targets europe beginning 2017. took advantage syrian military conﬂict thematic content ﬁle naming trump s_attack_on_syria_english.docx deployment likely part focus nato targets light splm deployment central asia consistent infrastructure meanwhile early-to-mid 2017 splm/chopstick/xagent detections central asia provided glimpse ongoing focus ex-soviet republics central asia particular detections interesting indicate attempted selective 2nd stage deployment backdoor maintaining ﬁlestealer keylogger remoteshell functionality asystem interest latest revision backdoor portions splm match previous reports splm/xagent similarities maintained splm 64-bit modules already appeared version software may year targeting proﬁles included defense related commercial military organizations telecommunications targeting included tr kz kg jo uk uz heavy zebrocy deployments since mid-november 2015 threat actor referred sofacy apt28 utilizing unique payload delivery mechanism written delphi autoit collectively refer package related activity zebrocy written reports usage development june 2017 sofacy developers modiﬁed redeployed incremented versions malware zebrocy chain follows pattern spearphish attachment compiled autoit script downloader zebrocy payload deployments observed sofacy actively developing deploying new package much smaller speciﬁc subset targets within broader set targeting proﬁles spearphish ﬁlenames lures carry thematic content related visa applications scanned images border control administration various administrative notes targeting appears widely spread across middle east europe asia /p style= margin- bottom:0 important business accounting practices standards science engineering centers industrial hydrochemical engineering standards/certiﬁcation ministry foreign aﬀairs embassies consulatesnational security intelligence agencies press services translation services ngo family social service ministry energy industry identiﬁed new msil components deployed zebrocy recent zebrocy versioning 7.1 related zebrocy modules drop ﬁle-stealing msil modules call covfacy v7.0 components unexpected inclusion particular toolset example one sent handful countries identiﬁes network drives added target systems rc4-like-encrypts writes certain ﬁle metadata contents local path later exﬁltration stealer searches ﬁles 60mb less extensions /p style= margin-bottom:0 important .doc .docx .xls .xlsx .ppt .pptx .exe .zip .rar execution installs application-deﬁned windows hook hook gets windows messages indicating network drive attached upon adding network drive hook calls recordtofile ﬁle stealer method zebrocy spearphishing targets af au az bd cn de es fi ge il jo kw kg kz lb lt mn nl om pk po sa za sk se ch tj tm tr ua uae uk us uz splm deployment central asia splm/chopstick components deployed throughout 2017 native 64-bit modular c++ windows com backdoors supporting http fully encrypted tlsv1 tlsv1.2 communications mostly deployed second half 2017 sofacy earlier splm activity deployed 32-bit modules unencrypted http sometimes smtp sessions 2016 saw fully functional large splm/x-agent modules supporting os x. executable module continues part framework supporting various internal external components communicating internal external channels maintaining slightly morphed encryption functionality per deployment sofacy selectively used splm/chopstick modules second stage implants high interest targets years change previous compilations module structured used inject remote shell keylogger ﬁlesystem add-ons processes running victim systems maintaining functionality originally present within main module newer splm modules deployed mostly central asian based targets may tie nato form targets include foreign aﬀairs government organizations localized abroad defense organizations presence localized located europe also located afghanistan one outlier splm target proﬁle within visibility includes audit consulting ﬁrm bosnia herzegovina minor changes updates code released deployments including new mutex format exclusive use encrypted http communications tls compiled code already altered per deployment multiple subtle ways order stymie identiﬁcation automated analysis accommodate targeted environments strings c2 domains functionality error messages etc custom encrypted per deployment targets tr kz ba tm af de lt nl splm/chop stick/xagent modularity infrastructurethis subset splm/chopstick activity leads several small surprises take us 2018 discussed detail sas 2018. group demonstrates malleability innovation maintaining producing familiar splm functionality pragmatic systematic approach towards producing undetected diﬃcult-to-detect malware continues changes second stage splm backdoor reﬁned making code reliably modular infrastructure notes sofacy set maintained multiple servers c2 varying durations registering fairly recognizable domains privacy services registrars accept bitcoin fake phone numbers phony individual names email address domain registration relationships activity patterns publicly disclosed expect see change process 2018. also throughout year previous years researchers began comment publicly sofacy fairly consistent infrastructure setup always attackers make mistakes give away hints providers registrars prefer interesting note version splm implements communications fully encrypted https example might see extraneous data ssl/tls certiﬁcates give away information provider resources leading summer 2017 infrastructure mostly created pdr internet domain service bs corp resellers hosting mostly provided fast serv inc resellers likelihood related bitcoin payment processing accordingly server side certiﬁcates appear generated locally vps hosts exclusively paid providers bitcoin merchant processing one certiﬁcate generated locally appeared hp-ux box another generated 8569985.securefastserver com email address root 8569985.securefastserver com seen nethostnet com domain certiﬁcate conﬁguration ignored malware addition ip data data point suggested qhoster https //www.qhoster com vps hosting reseller choice time noted reseller accepted alfa click paypal payza neteller skrill webmoney perfect money bitcoin litecoin solidtrust pay cashu ukash okpay egopay paysafecard alipay mg western union sofort banking qiwi bank transfer payment conclusion sofacy one active apt monitor continues spearphish way targets reportedly widely phishes credentials infrequently participates server side activity including host compromise beef deployment example ksn visibility detections suggests shift early 2017 high volume nato spearphish targeting towards middle east central asia ﬁnally moving focus east late 2017. operational security good campaigns appear broken subsets activity malware involving gamefish zebrocy splm name evolving modiﬁed splm/chopstick/xagent code long- standing part sofacy activity however much changing cover recent 2018 change targeting malware sas 2018. group like sofacy attention detected network important review logins unusual administrator access systems thoroughly scan sandbox incoming attachments maintain two factor authentication services like email vpn access order identify presence gain valuable insight targeting intelligence reports gain powerful means detections hunting tools like yara out-of-band processing solution like kata important technical appendix related md5 8f9f697aa6697acee70336f66f295837 1a4b9a6b321da199aa6d10180e889313 842454b48f5f80002 9946b1555fba7fc d4a5d44184333442f5015699c2b8af28 1421419d1be31f1f9ea60e8ed87277db b1d1a2c64474d2f6e7a5db71ccbafa31 953c7321c4959655fdd53302550ce02d 57601d717fcf358220340675f8d63c8a 02b79c468c38c4312429a499fa4f6c81 85cd38f9e2c9397a18013a8921841a04 f8e92d8b5488ea76c40601c8f1a08790 66b4fb539806ce27be184b6735584339 e8e1fcf757fe06be13bead43eaa1338c 953c7321c4959655fdd53302550ce02d aa2aac4606405d61c7e53140d35d7671 85cd38f9e2c9397a18013a8921841a04 57601d717fcf358220340675f8d63c8a 16e1ca26bc66e30bfa52f8a08846613d f8e92d8b5488ea76c40601c8f1a08790 b137c809e3bf11f2f5d867a6f4215f95 237e6dcbc6af50ef5f5211818522c463 88009adca35560810ec220544e4fb6aa 2163a33330ae5786d3e984db09b2d9d2 02b79c468c38c4312429a499fa4f6c81 842454b48f5f80002 9946b1555fba7fc d4a5d44184333442f5015699c2b8af28 b88633376fbb144971dcb503f72fd192 8f9f697aa6697acee70336f66f295837 b6f77273cbde76896a36e32b0c0540e1 1a4b9a6b321da199aa6d10180e889313 1421419d1be31f1f9ea60e8ed87277db 1a4b9a6b321da199aa6d10180e889313 9b10685b774a783eabfecdb6119a8aa3 aa34fb2e5849bﬀ4144a1c98a8158970 aced5525ba0d4f44ﬀd01c4db2730a34 b1d1a2c64474d2f6e7a5db71ccbafa31 b924ﬀ83d9120d934bb49a7a2e3c4292 cdb58c2999eeda58a9d0c70f910d1195 d4a5d44184333442f5015699c2b8af28 d6f2bf2066e053e58fe8bcd39cb2e9ad 34dc9a69f33ba93e631cd5048d9f2624 1c6f8eba504f2f429abf362626545c79 139c9ac0776804714ebe8b8d35a04641 e228cd74103dc069663bb87d4f22d7d5 bed5bc0a8aae2662ea5d2484f80c1760 8c3f5f1ﬀf999bc783062dd50357be79 5882a8dd4446abd137c05d2451b85fea 296c956fe429cedd1b64b78e66797122 82f06d7157dd28a75f1fbb47728aea25 9a975e0ddd32c0deef1318c485358b20 529424eae07677834a770aaa431e6c54 4cafde8fa7d9e67194d4edd4f2adb92b f6b2ef4daf1b78802548d3e6d4de7ba7 ede5d82bb6775a9b1659dccb699fadcb 116d2fc1665ce7524826a624be0ded1c 20ﬀ290b8393f006eaf4358f09f13e99 4b02dfdfd44df3c88b0ca8c2327843a4 c789ec7537e300411d523aef74407a5e 0b32e65caf653d77cab2a866ee2d9dbc 27faa10d1bec1a25f66e88645c695016 647edddf61954822ddb7ab3341f9a6c5 2f04b8eb993ca4a3d98607824a10acfb 9fe3a0fb3304d749aeed2c3e2e5787eb 62deab0e5d61d6bf9e0ba83d9e1d7e2b 86b607fe63c76b3d808f84969cb1a781 f62182cf0ab94b3c97b0261547dfc6cf 504182aaa5575bb38bf584839beb6d51 d79a21970cad03e22440ea66bd85931f related domains nethostnet com hostsvcnet com etcrem net movieultimate com newﬁlmts com fastdataexchange org liveweatherview com analyticsbar org analyticstest net lifeofmentalservice com meteost com righttopregnantpower com kiteim org adobe-ﬂash-updates org generalsecurityscan com globalresearching org lvueton com audiwheel com online-reggi com fsportal net netcorpscanprotect com mvband net mvtband net viters org treepastwillingmoment com sendmevideo org satellitedeluxpanorama com ppcodecs com encoder-info tk wmdmediacodecs com postlkwarn com shcserv com versiontask com share post webcdelivery com miropc org securityprotectingcorp com uniquecorpind com appexsrv net adobeupgradeﬂash com apt backdoor campaigns cyber espionage nation state sponsored espionage sofacy targeted attacks vulnerabilities exploits zero-day vulnerabilities related posts zero-day vulnerability telegramgas expensive let make cheap denis co"

In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    base_prompt.format(
        # INSTRUCTION, # instruction
        report, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Extract tactics from the following text. Tactics represent the "why" of an attack technique or sub-technique. It is a tactical goal or reason for performing an action. Choose one or more from these options\n    Reconnaissance, Resource Development, Initial Access, Persistence, Execution, Privilege Escalation, Defense Evasion, Credential Access, Discovery, Lateral Movement, Collection, Command and Control, Exfiltration, Impact \n    Extract tactics from the following text. \n### Text:\nsofacy also known apt28 fancy bear tsar team highly active proliﬁc apt high volume 0day deployment innovative broad malware set sofacy one top groups monitor report protect 2017 diﬀerent regard private reports subscription customers receive steady stream yara ioc reports sofacy reported apt year high level cyber-espionage activity goes back years 2011-2012 group used relatively tiny implant known sofacy sourface ﬁrst stage malware time similarities old miniduke implants made us believe 

# TO-DO:

maybe include this fragment in the prompt:


Tactics represent the "why" of an ATT&CK technique or sub-technique. It is the adversary's tactical goal: the reason for performing an action. For example, an adversary may want to achieve credential access

    Reconnaissance (Enterprise, ICS)
    Resource Development (Enterprise, ICS)
    Initial Access
    Execution
    Persistence
    Privilege Escalation
    Defense Evasion
    Credential Access (Enterprise, Mobile)
    Discovery
    Lateral Movement
    Collection
    Command and Control
    Exfiltration (Enterprise, Mobile)
    Impact
    Network Effects (Mobile-only)
    Network Service Effects (Mobile-only)
    Inhibit Response Function (ICS-only)
    Impair Process Control (ICS-only)